# 提示词里模板中的模板

## MessagePromptTemplate

LangChain提供了不同类型的 MessagePromptTemplate 。最常用的是 AIMessagePromptTemplate 、 SystemMessagePromptTemplate 和 HumanMessagePromptTemplate ，它们分别创建 AI 消息、系统消息和人工消息。

但是，如果聊天模型支持使用任意角色获取聊天消息，则可以使用 ChatMessagePromptTemplate ，它允许用户指定角色名称。

In [ ]:
from langchain_core.prompts import ChatMessagePromptTemplate

template = "愿{subject}与你同在！"

prompt = ChatMessagePromptTemplate.from_template(
    role="kkutys", template=template
)

print(prompt.format(subject="上帝"))

LangChain 还提供了 MessagesPlaceholder ，它使您可以完全控制格式化期间要呈现的消息。当您不确定消息提示模板应使用什么角色或希望在格式化期间插入消息列表时，这会很有用。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

human_prompt = "用{word_count}个字总结一下迄今为止我们之间的对话。"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"),
    human_message_prompt
])

human_message = HumanMessage(content="学习炒股最好的方法是什么？")
ai_message = AIMessage(
    content="""\
        1. 首先，学习各类金融知识，了解各类财务指标的含义。
        2. 其次，学习各类财报的阅读和分析，找到适合自己的一种的财报解读方法论。
        3. 最后，学习1-2两种技术指标，可以参考一些经典技术书籍学习，找到适合自己的一种组合指标分析方法。
    """
)

print(chat_prompt.format_messages(
    conversation=[human_message, ai_message], 
    word_count=10
))


In [16]:
# 结合大模型分析
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9, max_tokens=8192, model="glm-4-plus")

output_parser = StrOutputParser()

chain = chat_prompt | llm | output_parser

print(chain.invoke({"conversation": [human_message, ai_message], "word_count": 50}))

探讨炒股学习方法，建议先学金融知识、财报分析，再掌握技术指标，形成个人投资策略。


## 部分提示模板

与其他方法一样，“部分”提示模板是有意义的 - 例如传入所需值的子集，以创建一个仅需要剩余值子集的新提示模板。

LangChain 通过两种方式支持这一点： 
1. 使用字符串值进行部分格式化。 
2. 使用返回字符串值的函数进行部分格式化。

这两种不同的方式支持不同的用例。在下面的示例中，我们将回顾这两个用例的动机以及如何在 LangChain 中实现这一点。


### 部分带字符串

想要部分提示模板的一个常见用例是，如果您先于其他变量获取某些变量。例如，假设您有一个提示模板需要两个变量 foo 和 baz 。如果您在链的早期获得 foo 值，但稍后获得 baz 值，那么等到两个变量都位于同一位置才能将它们传递给提示模板。相反，您可以使用 foo 值部分化提示模板，然后传递部分化的提示模板并直接使用它。下面是执行此操作的示例：

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("foo: {foo} baz: {baz}")

partial_prompt = prompt.partial(foo="bar")

print(partial_prompt.format(baz="qux"))

In [ ]:
prompt = PromptTemplate(
    template="foo: {foo} baz: {baz}",
    partial_variables={"foo": "bar"},
    input_variables=["baz"]
)

print(prompt.format(baz="qux"))

## 部分函数

另一个常见用途是对函数进行部分处理。这样做的用例是当您有一个变量时，您知道您总是希望以通用方式获取该变量。一个典型的例子是日期或时间。想象一下，您有一个提示，您总是希望获得当前日期。您无法在提示中对其进行硬编码，并且将其与其他输入变量一起传递有点烦人。在这种情况下，能够使用始终返回当前日期的函数来部分提示是非常方便的。

In [ ]:
from datetime import datetime


def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

prompt = PromptTemplate.from_template("今天是{date}, 请给我讲一个关于{topic}的经典论述.")

partial_prompt = prompt.partial(date=get_current_date)

print(partial_prompt.format(topic="爱情"))

In [17]:
# 结合大模型分析
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os


prompt = PromptTemplate(
    template="今天是{date}, 请给我讲一个关于{topic}的经典论述.",
    input_variables=["topic"],
    partial_variables={"date": get_current_date}
)

load_dotenv()

api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.9, max_tokens=8192, model="glm-4-plus")

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

for s in chain.stream({"topic": "量子力学"}):
    print(s, end="", flush=True)

当然可以。量子力学是20世纪初发展起来的一门物理学分支，它彻底改变了我们对自然界的认识。其中一个经典的论述是由奥地利物理学家埃尔温·薛定谔提出的“薛定谔的猫”思想实验。

### 薛定谔的猫

**背景：**
薛定谔的猫是薛定谔在1935年提出的一个思想实验，旨在探讨量子力学中的叠加态和测量问题。这个实验通过一个极端的例子，揭示了量子力学与经典物理学之间的深刻矛盾。

**实验描述：**
1. **装置：** 一个封闭的箱子，里面有一只猫、一瓶毒气、一个放射性原子、一个盖革计数器和一个锤子。
2. **机制：**
   - 放射性原子有50%的概率在1小时内衰变。
   - 如果盖革计数器检测到原子衰变，锤子会砸碎毒气瓶，猫会被毒死。
   - 如果原子没有衰变，猫则存活。

**核心思想：**
根据量子力学的哥本哈根解释，在没有进行观察之前，放射性原子处于“衰变”和“未衰变”的叠加态。因此，箱子里的猫也处于“生”和“死”的叠加态。

**薛定谔的质疑：**
薛定谔通过这个思想实验提出了一个关键问题：量子力学中的叠加态在宏观世界中是否依然适用？在未打开箱子进行观察之前，猫究竟是生是死？这引发了关于量子力学测量问题的广泛讨论。

**意义：**
- **叠加态：** 强调了量子系统在未观测前处于多种可能状态的叠加。
- **测量问题：** 探讨了观察者在量子力学中的角色，即测量行为如何导致波函数坍缩。
- **量子与经典的界限：** 引发了对量子力学如何过渡到经典物理学的思考。

**后续影响：**
薛定谔的猫成为量子力学中最著名的思想实验之一，激发了无数科学家对量子力学基础问题的深入研究，包括多世界解释、量子退相干等理论。

这个思想实验不仅展示了量子力学的奇特性质，也揭示了我们对自然界最基本概念的深刻反思。希望通过这个经典论述，你能对量子力学的复杂性和魅力有更深的理解。